In [6]:
from wptherml.wpml import multilayer
from matplotlib import pyplot as plt
from wptherml.datalib import datalib
from wptherml.numlib import numlib
from scipy.optimize import minimize
from numpy import random
import time
import numpy as np
h = 6.626e-34
c = 299792458.
temp = 428
wl1 = 3000e-9
wl2 = 4200e-9

### define Lamba_List bounds
startwl = 300e-9
endwl = 20300e-9
numwl = 5001;

### create a dummy stack to get refractive indices
structure = {
        'Material_List' : ['Air', 'SiO2', 'Ta2O5', 'Air'],
        'Thickness_List': [0, 1e-7, 1e-7, 0 ],
        'Lambda_List': [startwl, endwl, numwl],
        'Temperature': temp
       }
### create multilayer
dummy = multilayer(structure)
lambdas = dummy.lambda_array

### find Lamba_List indices for wl1 and wl2
d_lamb = (endwl - startwl)/(numwl-1)
wl1i = round((wl1 - startwl)/(d_lamb))
wl2i = round((wl2 - startwl)/(d_lamb))

### get refractive index arrays
n_SiO2 = dummy.layer_ri(1)
n_Ta2O5 = dummy.layer_ri(2)

### calculate effective quarter wave lengths for given vacuum wavelengths
thickL1 = 0.25*wl1/n_SiO2[wl1i].real
thickL2 = 0.25*wl2/n_SiO2[wl2i].real
thickH1 = 0.25*wl1/n_Ta2O5[wl1i].real
thickH2 = 0.25*wl2/n_Ta2O5[wl2i].real

print("thickL1: ",thickL1)
print("thickL2: ",thickL2)
print("thickH1: ",thickH1)
print("thickH2: ",thickH2)

### create a full filter-PS-filter stack
full_structure = {
        'Material_List' : ['Air', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'PS', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Air'],
        'Thickness_List': [0, thickL1/2, 1.5e-9, thickL1/2, thickH1, thickL1, thickH1, 1e-4, 1e-3, thickL2/2, 1.5e-9, thickL2/2, thickH2, thickL2, thickH2, 1e-4, 0],
        'Lambda_List': [startwl, endwl, numwl],
        'Temperature': temp
       }

### create the bottom filter stack
structure_bottom = {
        'Material_List' : ['Air', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Air'],
        'Thickness_List': [0, thickL2/2, 1.5e-9, thickL2/2, thickH2, thickL2, thickH2, 1e-4, 0],
        'Lambda_List': [startwl, endwl, numwl],
        'Temperature': temp
       }
# create top filter
structure_top = {
        'Material_List' : ['Air', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Air'],
        'Thickness_List': [0, thickL1/2, 1.5e-9, thickL1/2, thickH1, thickL1, thickH1, 1e-4, 0],
        'Lambda_List': [startwl, endwl, numwl],
        'Temperature': temp
       }


### create multilayer
full = multilayer(full_structure)

### create multilayer
bottom = multilayer(structure_bottom)
top = multilayer(structure_top)



### Get the solar spectrum
AM = datalib.AM(full.lambda_array)

### Get the solar flux spectrum (number of photons per second per wavelength per meter squared)
AMflux = AM * full.lambda_array / (h * c)

### Integrate the above-gap solar flux transmitted through optic
AM_transmit = numlib.Integrate(AM * (1 - full.reflectivity_array), full.lambda_array, startwl, 760e-9)
print(AM_transmit," photons / s / m^2 transmitted")

### Integrate the total above-gap solar flux
AM_tot = numlib.Integrate(AM, full.lambda_array, startwl, 760e-9)
print(AM_tot," photons / s / m^2 total from Sun")

print(100* AM_transmit / AM_tot,"% transmissive")


### Make a BB spectrum at specified temp
rho = datalib.BB(full.lambda_array, temp)

### plot reflectivity and absorptivity/emissivity of the multilayer




  Gradient will be taken with respect to all layers! 
thickL1:  5.284521005201405e-07
thickL2:  7.602923357738591e-07
thickH1:  3.733648095151694e-07
thickH2:  5.311796118517261e-07
  Gradient will be taken with respect to all layers! 
  Gradient will be taken with respect to all layers! 
  Gradient will be taken with respect to all layers! 
354.6042379758658  photons / s / m^2 transmitted
472.21589043999995  photons / s / m^2 total from Sun
75.09366905155471 % transmissive


In [8]:
def update_bottom(rho):
    # fixed thicknesses
    full.d[1] = 1.5
    full.d[6] = 1e5
    full.d[7] = 1e6
    full.d[9] = 1.5
    full.d[14] = 1e5
    
    # random thicknesses
    full.d[2] = random.uniform(1, 800)
    full.d[3] = random.uniform(1, 800)
    full.d[4] = random.uniform(1, 800)
    full.d[5] = random.uniform(1, 800)
    full.d[8] = random.uniform(1, 800)
    full.d[10] = random.uniform(1, 800)
    full.d[11] = random.uniform(1, 800)
    full.d[12] = random.uniform(1, 800)
    full.d[13] = random.uniform(1, 800)
    
    for i in range(0, len(full.d)):
        full.d[i] *= 1e-9
    
    

    bottom.d[1:8] = full.d[9:16]
    top.d[1:8] = full.d[1:8]
    
    print("top")
    print(top.d)
    print("bottom")
    print(bottom.d)
    
    print("full")
    print(full.d)
    ### now we have the new structure, update fresnel quantities
    bottom.fresnel()
    top.fresnel()
    full.fresnel()


    ### now we have new emissivity, update thermal emission
    BB_blocked = numlib.Integrate(rho * bottom.reflectivity_array, bottom.lambda_array, 3500e-9, 20300e-9)
    ### Integrate the total BB radiation longer than 3.5 um
    BB_tot = numlib.Integrate(rho, full.lambda_array, 3500e-9, endwl)
    ref_tot = numlib.Integrate(full.reflectivity_array, full.lambda_array, 3500e-9, endwl)
    ref_bottom = numlib.Integrate(bottom.reflectivity_array, bottom.lambda_array, 3500e-9, endwl)
    BB_trapped = 0.5*numlib.Integrate(rho * bottom.reflectivity_array, bottom.lambda_array, 3500e-9, endwl) + 0.5*numlib.Integrate(rho * top.reflectivity_array, top.lambda_array, 3500e-9, endwl)
    ### Integrate the transmitted BB radiation shorter than 3.5 um
    EBE_transmitted = numlib.Integrate(rho * (1 - bottom.reflectivity_array), bottom.lambda_array, 1000e-9, 3500e-9)
    ### Integrate the total BB radiation shorter than 3.5 u
    EBE_tot = numlib.Integrate(rho, full.lambda_array, 1000e-9, 3500e-9)
    print("BB_blocked: ",BB_blocked/BB_tot * 100)
    print("BB_trapped: ",BB_trapped/BB_tot * 100)
    print("EBE_transmitted: ",EBE_transmitted/EBE_tot * 100)
    return -0.1 * BB_blocked/BB_tot - 0.1 * BB_trapped/BB_tot - 0.8 * EBE_transmitted/EBE_tot

In [9]:
update_bottom(rho)

top
[0, 1.5000000000000002e-09, 1.4704781456687105e-07, 2.712972447354631e-07, 2.804382207091503e-07, 3.247249733257897e-07, 0.0001, 0.001, 0]
bottom
[0, 1.5000000000000002e-09, 7.261642683241869e-07, 4.151517170721168e-07, 4.0351371149141634e-07, 6.295587130756535e-07, 0.0001, 1.0000000000000002e-13, 0]
full
[0.0, 1.5000000000000002e-09, 1.4704781456687105e-07, 2.712972447354631e-07, 2.804382207091503e-07, 3.247249733257897e-07, 0.0001, 0.001, 5.955573270395524e-07, 1.5000000000000002e-09, 7.261642683241869e-07, 4.151517170721168e-07, 4.0351371149141634e-07, 6.295587130756535e-07, 0.0001, 1.0000000000000002e-13, 0.0]


/opt/anaconda3/envs/wptbeta/lib/python3.7/site-packages/wptherml/tmm.py:151: RuntimeWarning: overflow encountered in matmul
  t2 = np.matmul(t1,Pl)
/opt/anaconda3/envs/wptbeta/lib/python3.7/site-packages/wptherml/tmm.py:150: RuntimeWarning: invalid value encountered in matmul
  t1 = np.matmul(M,Dl)
/opt/anaconda3/envs/wptbeta/lib/python3.7/site-packages/wptherml/tmm.py:150: RuntimeWarning: overflow encountered in matmul
  t1 = np.matmul(M,Dl)


BB_blocked:  98.70667638472315
BB_trapped:  nan
EBE_transmitted:  1.9282485241296037


nan